Exercice 1 : Entropie

In [3]:
import decisiontree as dt
import math
import collections

def entropie(vect):
    """
    calcule l'entropie d'un objet itérable
    """
    entropy = 0
    labels_dict = collections.Counter(vect)
    for k,v in labels_dict.items():
        entropy-= v/len(vect) * math.log(v/len(vect))
    return entropy    

def entropie_cond(list_vect):
    """
    calcule l'entropie conditionnelle d'une liste d'itérables
    """
    h, nb_elems = 0,0
    for vect in list_vect:
        h += len(vect) * entropie(vect)
        nb_elems += len(vect)
    return h/nb_elems


Question 1.3:

In [4]:
import  pickle
import  numpy as np

# data : tableau (films ,features), id2titles : dictionnaire  id -> titre ,
# fields : id  feature  -> nom
[data , id2titles , fields ]= pickle.load(open("imdb_extrait.pkl","rb"))

# la  derniere  colonne  est le vote
datax=data [: ,:32]
datay=np.array ([1 if x[33] >6.5  else  -1 for x in data])

entropies = [entropie(datax[i]) for i in range(len(datax[0]))]
print(entropies)
print(fields)
#entropies_cond = [entropie_cond(datax[i]) for i in range(len(datax[0]))]
#print(entropies_cond)

#diff_entropies = entropies - entropies_cond



[0.8479942625588093, 0.8479942625588093, 0.6848735471085738, 0.8479942625588093, 0.9606247612828218, 0.8479942625588093, 0.8479942625588093, 0.8479942625588093, 0.9089087348987808, 0.715537563594637, 0.8479942625588093, 0.8479942625588093, 0.9089087348987808, 0.8479942625588093, 0.7752107668722757, 0.6848735471085738, 0.9089087348987808, 0.9089087348987808, 0.9089087348987808, 0.9089087348987808, 0.8479942625588093, 0.8479942625588093, 0.8479942625588093, 0.9606247612828218, 0.9606247612828218, 0.8479942625588093, 0.7752107668722757, 0.7752107668722757, 0.8479942625588093, 0.8479942625588093, 0.8479942625588093, 0.6848735471085738]
{0: 'Sci-Fi', 1: 'Crime', 2: 'Romance', 3: 'Animation', 4: 'Music', 5: 'Comedy', 6: 'War', 7: 'Horror', 8: 'Film-Noir', 9: 'Adult', 10: 'News', 11: 'Reality-TV', 12: 'Thriller', 13: 'Western', 14: 'Mystery', 15: 'Short', 16: 'Talk-Show', 17: 'Drama', 18: 'Action', 19: 'Documentary', 20: 'Musical', 21: 'History', 22: 'Family', 23: 'Adventure', 24: 'Fantasy', 

In [5]:

from  decisiontree  import  DecisionTree
depths = [i for i in range(31)]
scores = []
for depth in depths:
    dt = DecisionTree ()
    dt.max_depth = depth#on fixe la  taille  de l’arbre a 5
    dt.min_samples_split = 2#nombre  minimum d’exemples  pour  spliter  un noeud
    dt.fit(datax ,datay)
    dt.predict(datax [: ,:])
    print(dt.score(datax ,datay))
    scores.append(dt.score(datax ,datay))

# dessine l’arbre  dans un  fichier  pdf   si pydot  est  installe.
#dt.to_pdf("/tmp/test_tree.pdf",fields)
# sinon  utiliser  http :// www.webgraphviz.com/
#dt.to_dot(fields)
#ou dans la  console
#print(dt.print_tree(fields ))

0.5672552866797471
0.6418138216699367
0.6825812077610639
0.7183344233703945
0.7314148681055156
0.736429038587312
0.7575757575757576
0.7719642467843907
0.7889688249400479


KeyboardInterrupt: 